# Checking enviroment states and action sets

In [1]:
import sys
import os.path
sys.path.append("..")
from PPO_Agent_with_intrinsic_reward import * #PPO_Agent_v2 PPO_Agent_with_Guided_AC
from ENV_DETAILS import *

2023-12-15 16:39:24.241611: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num devices available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
if os.name == 'nt':
    main_hyper_dir = "D:\\Artificial_Intelligence\\Portfolio\\RL_updated\\Pendulum\\" # Windows
    conda_python_exec = 'C:\\Users\\yanie\\anaconda3\\envs\\ai_dev\\python.exe '# Windows
    logs_dir = main_hyper_dir+'Hyperparam_tuning\\' # Windows
else:
    main_hyper_dir = "/media/n/NewDisk/Artificial_Intelligence/Portfolio/RL_updated/Pendulum/" # Linux
    conda_python_exec = '/home/n/anaconda3/envs/ai_dev/bin/python '# Linux
    logs_dir = main_hyper_dir+'Hyperparam_tuning_a3c/' # Linux

ENV = "Pendulum-v1"
SUCESS_CRITERIA_VALUE = ENV_DETAILS[ENV]["SUCESS_CRITERIA_VALUE"]
SUCESS_CRITERIA_EPOCH = ENV_DETAILS[ENV]["SUCESS_CRITERIA_EPOCH"]
EPISODES = ENV_DETAILS[ENV]["EPISODES"]

In [3]:

env = gym.make(ENV)
env

<TimeLimit<OrderEnforcing<PendulumEnv<Pendulum-v1>>>>

In [4]:
env._max_episode_steps

200

In [5]:
s = env.observation_space.sample()
s

array([-0.08234285, -0.23212363,  4.406051  ], dtype=float32)

In [6]:
env.observation_space, env.observation_space.shape[0]

(Box([-1. -1. -8.], [1. 1. 8.], (3,), float32), 3)

In [7]:
s = env.reset()
s[0]

0.96322674

In [8]:
env.action_space

Box(-2.0, 2.0, (1,), float32)

In [9]:
s = env.reset()
env.action_space, env.action_space, env.step(s)

(Box(-2.0, 2.0, (1,), float32),
 Box(-2.0, 2.0, (1,), float32),
 (array([-0.46611664,  0.88472325,  0.5077581 ], dtype=float32),
  -4.123292908600494,
  False,
  {}))

### Training/Hyperparam run

In [10]:
%tensorboard --logdir logs_general/hyper

UsageError: Line magic function `%tensorboard` not found.


In [11]:
TUNING_TYPE = "BAYES"
HYPERPARAM_TUNING = True
writer= "Training/fit_PPO/"

In [12]:
if HYPERPARAM_TUNING:

    dir = r"Hyperparam_kt_ppo"
    project_name = "keras_tunning_ppo"

    tuner = kt.BayesianOptimization(
            MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/ppo/", evaluation_epoch = env._max_episode_steps, training_steps = 700000,
                sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value= SUCESS_CRITERIA_VALUE,
                discount_min = 0.95, discount_max = 0.99,
                gae_min = 0.94, gae_max = 0.96,
                lr_actor_min = 0.00001, lr_actor_max = 0.001,
                lr_critic_min = 0.00001, lr_critic_max = 0.001,
                entropy_factor_min = 0.01, entropy_factor_max = 0.1,
                dense_min = 32, dense_max = 256,
                environment_name=ENV,
                reward_scaler = 1, num_layers_act = 2, num_layers_crit =2, num_layers_model = 2, training_epoch = 50, memory_size= env._max_episode_steps),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 50,
            # distribution_strategy= strategy,
            directory=dir,
            project_name=project_name
        )
    tuner.search(x=[0], y=[1])
else : 
    
        print("Acquiring parameters ....")
        writer= "Training/fit_ppo/"

        training_steps = 1000000
        entropy_factor = 0.05
        discount = 0.99
        dense_units_actor = [128]#64, 32]
        num_layers_actor = 1
        dense_units_critic = [128]#64,32]
        num_layers_crit =1
        num_layer_m = 1
        dense_units_model = [32]

        model = run_training(training_steps,  discount, dense_units_actor,  dense_units_critic, dense_units_model,  num_layers_actor, num_layers_crit, num_layer_m, writer, 
                      environment_name = ENV,reward_scaler = 1, return_agent = True, lr_actor= 0.00001, lr_critic= 0.00001, 
                      gae_lambda = 0.95, entropy_coeff = entropy_factor, policy_clip = 0.2, training_epoch = 50)


Trial number :  0

Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
0.97              |0.97              |discount
0.94              |0.94              |gae_lambda
0.00084088        |0.00084088        |lr_actor
0.0008108         |0.0008108         |lr_critic
0.00099861        |0.00099861        |lr_model
0.1               |0.1               |policy_clip
0.31064           |0.31064           |scaling_factor_reward
0.012824          |0.012824          |entropy_coeff
2                 |2                 |n_dense_layers_actor
252               |252               |dense_units_act_0
2                 |2                 |n_dense_layers_critic
187               |187               |dense_units_crit_0
1                 |1                 |n_dense_layers_model
64                |64                |n_dense_layers_model0

Trial number :  1
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn

In [ ]:
exploration_tech = "soft"
hyperparam_combination=[]

for trials in tuner.oracle.get_best_trials(num_trials=50):
    if int(trials.trial_id) == 49:
        print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

In [ ]:
val_env = gym.make(ENV, render_mode = "rgb_array")
dir = r"Hyperparam_kt_ppo"

for trials in tuner.oracle.get_best_trials(num_trials=1):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

    training_steps = 1000000
    entropy_factor = trials.hyperparameters.values["entropy_coeff"]
    discount = trials.hyperparameters.values["discount"]
    dense_units_actor = [128]#64, 32]
    num_layers_actor = 1
    dense_units_critic = [128]#64,32]
    num_layers_crit =1

    model = run_training(training_steps,  discount, dense_units_actor,  dense_units_critic, num_layers_actor, num_layers_crit, writer, 
                    environment_name = ENV,reward_scaler = 1, return_agent = True, lr_actor= 0.00001, lr_critic= 0.00001, 
                    gae_lambda = 0.95, entropy_coeff = entropy_factor, policy_clip = 0.2, training_epoch = 50)

    
env_model = tuner.get_best_models()[0]
final_rewards = final_evaluation(env_model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./ppo_"+exploration_tech+"_video.mp4")
print("Final mean reward '",exploration_tech,"':", np.mean(final_rewards))